Credit:

https://www.kaggle.com/gemartin/load-data-reduce-memory-usage

https://www.kaggle.com/carlmcbrideellis/jane-street-eda-of-day-0-and-feature-importance

https://www.kaggle.com/columbia2131/speed-up-reading-csv-to-pickle

In [ ]:
# numpy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import datatable as dt

# garbage collector to keep RAM in check
import gc  

# system
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

# 1. Load data
#### The original train.csv is large: 18.557G with 3141411 rows. We should not use **pandas** to load this file. Indeed, we should use **datatable** to avoid OOM issue and speed up the loading process then convert the loaded data to pandas dataframe.

In [ ]:
%%time
train_data = dt.fread('../input/ubiquant-market-prediction/train.csv')

In [ ]:
%%time
train_data = train_data.to_pandas()

# 2. Reduce MEM usage

In [ ]:
%%time
train_data = reduce_mem_usage(train_data)

# 3. Save the reduced dataframe

In [ ]:
train_data.to_pickle('train.pkl')